In [1]:
import cirq
import cirq_google as cg
import torch
import numpy as np
from shallow_cirq_6x6_pink import get_circuit, get_two_qubits_6x6

In [2]:
# # Set up the real engine and sampler on Willow Delphi
# run_name = 'Feb_1_cz_calibration'
# eng = cg.get_engine('serious-cat-404923')
# sampler_real = eng.get_sampler(processor_id='WIL01_3A_DELPHI', run_name=run_name, device_config_name='cz_71')
# Set up the real engine and sampler on Willow Delphi
#snapshot_id = '2025-02-21_234000.180'
snapshot_id = '2025-04-03_092452.945'
eng = cg.get_engine('serious-cat-404923')
sampler_real = eng.get_processor('WLA1HHPR00V02_4A_PINK').get_sampler(device_config_name='d7', snapshot_id=snapshot_id)

In [2]:
# run_name = 'Feb_21_cz_calibration'
run_name = 'Feb_24_cz_calibration'
eng = cg.get_engine('serious-cat-404923')
sampler_real = eng.get_sampler(processor_id='WLA1HHPR00V02_4A_PINK', run_name=run_name, device_config_name='d7')

In [3]:
# Set up simulate sampler
sampler_sim = cirq.Simulator()

In [3]:
# test Bell state
qubits = [cirq.GridQubit(4,4), cirq.GridQubit(4,5)]
cir = cirq.Circuit(cirq.H.on_each(*qubits), cirq.CZ(*qubits), cirq.H(qubits[1]), cirq.M(*qubits, key='m'))
result = sampler_real.run(cir, repetitions=1000)
result.histogram(key='m')

Counter({3: 513, 0: 478, 1: 5, 2: 4})

### Two probe qubits experiment

#### Get qubits

In [4]:
#size = '3x3'
#size = '5x5'
size = '6x6'
qubits_matrix, probe_qubits, anc_pairs, all_qubits = get_two_qubits_6x6(d=5)
print(len(all_qubits))
circ = get_circuit(qubits_matrix, theta=0, phi=0, probe_qubits=probe_qubits, basis=[1,1], anc_pairs=anc_pairs)

60


In [5]:
probe_qubits

[cirq.GridQubit(3, 4), cirq.GridQubit(3, 8)]

#### Collect data

In [7]:
# collect data without randomized compiling
sampler = sampler_real
theta_range = np.linspace(0, np.pi/2, 11)
phi = np.pi*(5/4)
repetitions = 10**5
folder_name = snapshot_id

for d in [5]: 
    qubits_matrix, probe_qubits, anc_pairs, all_qubits = get_two_qubits_6x6(d=d)
    for theta_idx in [8,9,10]:
        for i in [1]:
            for b1 in range(3):
                for b2 in range(3):
                    circ = get_circuit(qubits_matrix, theta=theta_range[theta_idx], phi=phi, probe_qubits=probe_qubits, basis=[b1,b2], anc_pairs=anc_pairs)
                    circ.append(cirq.measure(*all_qubits, key='m'))
                    result = sampler.run(circ, repetitions=repetitions)
                    print(f'distance={d}, loop={i}, size={size}, theta={theta_idx}, basis={b1},{b2}')
                    measurements = torch.tensor(result.measurements['m']).to(int)
                    torch.save(measurements, f'data/{folder_name}_d={d}/theta{theta_idx}/loop{i}/theta={theta_idx}_({b1},{b2}).pt') # save sequence

distance=5, loop=1, size=6x6, theta=10, basis=0,0
distance=5, loop=1, size=6x6, theta=10, basis=0,1
distance=5, loop=1, size=6x6, theta=10, basis=0,2
distance=5, loop=1, size=6x6, theta=10, basis=1,0
distance=5, loop=1, size=6x6, theta=10, basis=1,1
distance=5, loop=1, size=6x6, theta=10, basis=1,2
distance=5, loop=1, size=6x6, theta=10, basis=2,0
distance=5, loop=1, size=6x6, theta=10, basis=2,1
distance=5, loop=1, size=6x6, theta=10, basis=2,2


In [6]:
# collect data with randomized compiling
sampler = sampler_real
theta_range = np.linspace(0, np.pi/2, 11)
phi = np.pi*(5/4)
repetitions = 10**6
num_rc_circuits = 20
folder_name = snapshot_id

for d in [5]: 
    qubits_matrix, probe_qubits, anc_pairs, all_qubits = get_two_qubits_6x6(d=d)
    for theta_idx in [8,9,10]:
        for i in range(5):
            for b1 in range(3):
                for b2 in range(3):
                    circ = get_circuit(qubits_matrix, theta=theta_range[theta_idx], phi=phi, probe_qubits=probe_qubits, basis=[b1,b2], anc_pairs=anc_pairs)
                    circ.append(cirq.measure(*all_qubits, key='m'))
                    circuit_batch = [cirq.transformers.gauge_compiling.CZGaugeTransformer(circ) for _ in range(num_rc_circuits)]
                    circuit_batch = [cirq.merge_single_qubit_moments_to_phxz(circuit) for circuit in circuit_batch]
                    result = sampler.run_batch(circuit_batch, repetitions=int(repetitions/num_rc_circuits))
                    print(f'distance={d}, loop={i}, size={size}, theta={theta_idx}, basis={b1},{b2}')
                    measurements = torch.cat([torch.tensor(r[0].measurements['m']).to(int) for r in result])
                    torch.save(measurements, f'data/{folder_name}_d={d}/theta{theta_idx}/loop{i}/theta={theta_idx}_({b1},{b2}).pt') # save sequence

distance=5, loop=0, size=6x6, theta=8, basis=0,0
distance=5, loop=0, size=6x6, theta=8, basis=0,1
distance=5, loop=0, size=6x6, theta=8, basis=0,2
distance=5, loop=0, size=6x6, theta=8, basis=1,0
distance=5, loop=0, size=6x6, theta=8, basis=1,1
distance=5, loop=0, size=6x6, theta=8, basis=1,2
distance=5, loop=0, size=6x6, theta=8, basis=2,0
distance=5, loop=0, size=6x6, theta=8, basis=2,1
distance=5, loop=0, size=6x6, theta=8, basis=2,2
distance=5, loop=1, size=6x6, theta=8, basis=0,0
distance=5, loop=1, size=6x6, theta=8, basis=0,1
distance=5, loop=1, size=6x6, theta=8, basis=0,2
distance=5, loop=1, size=6x6, theta=8, basis=1,0
distance=5, loop=1, size=6x6, theta=8, basis=1,1
distance=5, loop=1, size=6x6, theta=8, basis=1,2
distance=5, loop=1, size=6x6, theta=8, basis=2,0
distance=5, loop=1, size=6x6, theta=8, basis=2,1
distance=5, loop=1, size=6x6, theta=8, basis=2,2
distance=5, loop=2, size=6x6, theta=8, basis=0,0
distance=5, loop=2, size=6x6, theta=8, basis=0,1
distance=5, loop=2, 

In [36]:
measurements.shape

torch.Size([100000, 60])

In [ ]:
print("Circuit depth:", len(circ))
#print(circ)

In [ ]:
all_qubits

In [ ]:
print(all_qubits[0], all_qubits[2], all_qubits[8])
print(all_qubits[-1], all_qubits[-4], all_qubits[-9])

In [ ]:
print((result.measurements['m'][:,-1]==result.measurements['m'][:,-9]).mean())
